# LCPA 23-24 - "Hierarchical mergers of binary black holes"

<center><h1>Group 07</h1></center>


<center><style>
table {
    font-size: 24px;
}
</style></center>

| Last Name          | First Name            |Student Number|
|--------------------|-----------------------|----------------|
| Bertinelli         | Gabriele              |2103359         |
| Boccanera          | Eugenia               |2109310         |
| Cacciola           | Martina               |2097476         |
| Lovato             | Matteo                |                |           

## 1. Introduction

A binary black hole (BBH) can form via close encounters of black holes (BHs) in a dense stellar environment, such as a nuclear star cluster (NSC), a globular cluster (GC) or a young star cluster (YSC). NSCs are very massive (~ $10^5 - 10^8 \, M_\odot$) star clusters lying at the center of some galaxies, including the Milky Way. GCs are old (~ 12 Gyr) massive (~ $10^4 - 10^6 M_\odot$) stellar clusters lying in the halo of galaxies. YSC are young (< 100 Myr) stellar clusters forming mostly in the disk of a galaxy.  

Several channels can lead to the formation of BBHs. But the distinctive signatures of the dynamical scenario is the formation of hierarchical mergers (IMs), i.e. repeated mergers of stellar-origin BHs that build up more massive one. This process is possible only in dense star clusters, where the merger remnant, which is initially a single BH, can pair up by dynamical exchanges or three-body encounter. The main obstacle to the formation of second-generation BHs via hierarchical mergers is the relativistic kick that the merger remnant receives at birth. This kick can be up to several thousand km/s. Hence, the interplay between the properties of the host star cluster (e.g., its escape velocity), those of the first-generation BBH population and the magnitude of the kick decides the maximum mass of a merger remnant in a given environment.  

A property that is being studied is that IM can build up IMBHs and also partially fill the pair instability (PI) mass gap between ~60 and ~120 $M_\odot$, explaining the formation of BBHs like GW190521.

#### Hierarchical mergers
When two stellar-born BHs merge via GW emission, their merger remnant is called second-generation (2g) BH. The 2g BH is a single object at birth. However, if it is retained inside its host star cluster, it may pair up dynamically with another BH. This gives birth to what we call a second-generation (2g) BBH, i.e. a binary black hole that hosts a 2g black hole . If a 2g binary black hole merges again, it gives birth to a third-generation (3g) BH, and so on. In this way, repeated black hole mergers in star clusters can give birth to hierarchical chains of mergers, leading to the formation of more and more massive black holes.

## 2. Goal of the project

Understand the differences between hierarchical binary black hole mergers in NSCs, GCc and YSc, by looking at a set of simulated BBHs. 
Our analysis will be carried out with classification ML algorithms, such as Random Forest and XGBoost. We will then proceed to analyze the importance of features in order to understand the properties of systems of BBHs.   

The idea is to split the analysis into two parts:

- Based on the features of the BBHs systems, figure out to which host star cluster these systems belong. 
  It's a classification problem with label the `label` column of the dataset corresponding to `0 -> GC, 1 -> NSC, 2 -> YSC`. Feature importance analysis will tell us which features are most important to understand which system belongs to which host stellar cluster.

- Analyze each stellar cluster independently. To do this we added a new label column `label_ngen`: `0` if the system has no other mergers beyond the 2nd generation; `1` if the system evolves beyond the 2nd generation.
  This is still a classification problem, this time with respect to `label_ngen`. Analysis of features importance will tell us which features are most important that lead systems to evolve and which do not.

To do this in a more detailed way, both globally (all labels together) and locally (single label), we will use `SHAP` values (Section ).

In order to have a cleaner notebook we created a file (`hmbh.py`) containing the functions needed to create the dataset, to train the ML models and to plot the results.

# 3. Dataset

The dynamical simulation were run for each host star cluster, with 12 different metallicities each. The files are found in the folders and subfolders: 

```python
folder = ['GC_chi01_output_noclusterevolv', 'NSC_chi01_output_noclusterevolv',
               'YSC_chi01_output_noclusterevolv']
    
metallicity = ['0.0002', '0.002', '0.02', '0.0004', '0.004', '0.006', '0.0008',
                   '0.008', '0.0012', '0.012', '0.0016', '0.016']
```

Each `nth_generation.txt` dataset is composed of 28 columns. Among them we selected these: 

```python
cols = ['c0', 'c1', 'c2', 'c3', 'c4', 'c7', 'c8', 'c9', 'c13', 'c15', 'c16', 'c17', 'c25', 'c27']
```

- c0: Identifier of the binary
- c1: Mass of the primary black hole in solar masses
- c2: Mass of the secondary black hole
  
- c3: Dimensionless spin magnitude of the primary black hole
- c4: Dimensionless spin magnitude of the secondary black hole
- c7: Initial semi-major axis of the binary black hole in solar radii
- c8: Initial orbital eccentricity of the binary black hole

- c9: Time requested for the dynamical pair up of the system in Myr
- c13: Time elapsed since the first-gen formation until the merger of the nth-gen system

- c15: Mass of the remnant
- c16: Dimensionless spin magnitude of the remnant black hole
- c17: Escape velocity from star cluster in km/s
- c25: Total mass of the stellar cluster
- c27: Number of generation of the system

We added two new columns, in the creation of the dataset:
- `label`: Identifies to which host star cluster a system belongs `0 -> GC, 1 -> NSC, 2 -> YSC`
- `label_ngen`: `0` if the system has no other mergers beyond the 2nd generation; `1` if the system evolves beyond the 2nd generation

### Import modules

In [ ]:
import hmbh as h # custom module for data analysis

import numpy as np
import pandas as pd
import polars as pl

import matplotlib as plt
import seaborn as sns

## 3.1 Dataset creation

In order to create the dataset we use the custom function `create_dataset`. It used the `polars.DataFrame` object. 
Polars is fast DataFrame library for manipulating structured data. In order to be fast it optimizes queries to reduce unneeded work/memory allocations and adheres to a strict schema (data-types should be known before running the query). 
Polars leverages the concept of `LazyFrame`, Polars doesn't run each query line-by-line but instead processes the full query end-to-end. This lazy evaluation strategy allows for better memory management, making it possible to work with datasets that are too large to fit into RAM (it inherits the concept of "lazy" from Dask).

The speed-up is astonishing: an improvement of ~ 900% wrt using Pandas!

In the creation of the dataset we filter out those systems that take longer than Hubble time (~ 13.6 Gyr) to merge.
After creating the dataset, we rename the columns and we add the column `label_ngen`.

In [ ]:
path = '../data/'

folder = ['GC_chi01_output_noclusterevolv', 'NSC_chi01_output_noclusterevolv',
               'YSC_chi01_output_noclusterevolv']
    
metallicity = ['0.0002', '0.002', '0.02', '0.0004', '0.004', '0.006', '0.0008',
                   '0.008', '0.0012', '0.012', '0.0016', '0.016']

cols = ['c0', 'c1', 'c2', 'c3', 'c4', 'c7', 'c8', 'c9', 'c13', 'c15', 'c16', 'c17', 'c25', 'c27'] # select bold columns -> most important for this analysis

new_cols = ['ID', 'bh_mass1', 'bh_mass2', 'spin1', 'spin2', 'semimajor', 'i_ecc', 'time_dyn', 'time_merge', 
'remnant_mass', 'remnant_spin', 'escape_vel', 'cluster_mass', 'n_gen']


df = h.create_dataset(path, folder, metallicity, cols) # dataset creation

df = h.rename_columns(df, new_cols) # rename columns

df = h.get_label_ngen(df) # get label for n_gen

In [ ]:
df.head(5)

# 4. Visual description
Below we plot some important features and we provide a description of their distribution and relation with the host star cluster and the number of generations.

### 4.1 Masses

# 5. Classification analysis

In this section we describe the machine learning tasks and their results.

### 5.1 Data preprocessing

`h.data_preprocessing`

'+' 

model_evaluation w/o bar_plot to show difference in label performance.